In [1]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
import openpyxl
import geopandas as gpd
import math

In [21]:
input_path = '../../data/input/dhis2/new_system/'
output_path = '../../data/output/sprint3_analysis/'
shapes_path = '../../data/shapes/district/districts_17_19.shp'
input_path_old = '../../data/input/dhis2/old_system/'
facility_path = '../../data/input/hospitals/original_data/'

# Some functions I'll use

In [3]:
districts = list(pd.read_csv(input_path+'districts.csv')["Districts"])
shapes = gpd.read_file(shapes_path)

# Creating a dict of names to replace dsirtcit names

def get_district_name_dict (df):
    keys = list(set(df.index).difference(set(shapes['name_19'])))
    values = list(set(shapes['name_19']).difference(set(df.index)))
    district_name = dict(zip(keys, values))
    return district_name 


In [4]:
# build a small fucntion to split the string column name of the data download as pivot 

def split(strng, sep, occ):
    strng = strng.split(sep)
    return sep.join(strng[occ[1]:]), sep.join(strng[:occ[0]]), sep.join(strng[occ[0]:occ[1]])[:3]

In [51]:
# To clean the data downloaded in a pivot format

def get_clean_pivot(df):
    df['facility']=df['orgunitlevel5']
    df['district']=df['orgunitlevel3'].apply(lambda x: x[:-9].upper())
    df['district'].rename(get_district_name_dict(df),inplace=True)
    cols = np.arange(0,9)
    df.drop(df.columns[cols],axis=1,inplace=True)
    cols = df.columns
    new_cols=[]
    for col in cols:
        new_cols.append(split(col,' ',[-2,-1]))
    df.columns=pd.MultiIndex.from_tuples(new_cols)
    return df

In [6]:
# To get year on year deltas over a list of months

def get_delta(df,st_year,end_year,indic,st_month,end_month):
    delta=((df[(end_year,indic,end_month)]
            -df[(st_year,indic,st_month)])
           /df[(st_year,indic,st_month)])
    return delta

In [7]:
# To get month on month deltas

def get_yoy_delta(df,indic,period):
    tot_18 = tot_19= tot_20 = np.zeros(135)
    for month in period:
        tot_18= tot_18+list(df[('2018',indic,month)])
        tot_19= tot_19+list(df[('2019',indic,month)])
        tot_20= tot_20+list(df[('2020',indic,month)])
    delta =  (tot_20-tot_19)/tot_19
    return tot_18,tot_19,tot_20,delta

# Look at SAM data

In [34]:
lbw_indic=['NUT: Percentage of children/babies born with low birth weight (<2.5kg)']

In [89]:
lbw_new=pd.read_csv(input_path+'lbw_facility_pivot.csv')
lbw_new=get_clean_pivot(lbw_new)

In [90]:
dropped_months=['Nov','Dec']
lbw_new.drop(dropped_months,axis=1,inplace=True,level=2)

# Look at all facilities

In [91]:
lbw_rep_all=lbw_new.copy().groupby('district').count()

C:\Users\Dalberg\anaconda3\envs\icohs\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [132]:
lbw_rep_all.sum()

2020      NUT: Percentage of children/babies born with low birth weight (<2.5kg)  Jan    1203.000000
                                                                                  Feb    1257.000000
                                                                                  Mar    1287.000000
                                                                                  Apr    1316.000000
facility                                                                                 7636.000000
2020      lbw reporting                                                           Jan      32.846248
                                                                                  Feb      33.277567
                                                                                  Mar      35.317479
                                                                                  Apr      36.065185
dtype: float64

In [92]:
# Should buid a function out of that 

for m in ['Jan','Feb','Mar','Apr']:
    lbw_rep_all[('2020','lbw reporting',m)]=lbw_rep_all[('2020','NUT: Percentage of children/babies born with low birth weight (<2.5kg)',m)]/lbw_rep_all['facility']

In [93]:
lbw_rep_all.sum(axis=0)/135

2020      NUT: Percentage of children/babies born with low birth weight (<2.5kg)  Jan     8.911111
                                                                                  Feb     9.311111
                                                                                  Mar     9.533333
                                                                                  Apr     9.748148
facility                                                                                 56.562963
2020      lbw reporting                                                           Jan     0.243306
                                                                                  Feb     0.246500
                                                                                  Mar     0.261611
                                                                                  Apr     0.267150
dtype: float64

# Look at Gvt facilites only

In [121]:
facility = pd.read_csv(facility_path+'MFL-Dec-19.csv')

In [122]:
facility['index_code']=facility['District'].apply(lambda x: x[:-9].upper())+facility['Health Facility']
facility.set_index('index_code',drop=True,inplace=True)

In [123]:
facility_gov=facility[facility['Ownership']=='GOV'].copy()

In [125]:
len(list(set(facility_gov.index).difference(set(lbw_new.index))))


96

In [126]:
lbw_new['index_code']=lbw_new['district']+lbw_new['facility']
lbw_new.set_index('index_code',drop=False,inplace=True)

In [127]:
lbw_gvt=pd.merge(facility_gov,lbw_new,how='left',left_index=True,right_index=True)

C:\Users\Dalberg\anaconda3\envs\icohs\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 3 on the right)
  warnings.warn(msg, UserWarning)


Paused here

In [129]:
lbw_gvt
#multi_index=pd.MultiIndex.from_tuples([('Test','test','test')])
#data_elements=pd.DataFrame(index=districts,columns=multi_index)

,Region,District,Sub county,Sub county UID,Health Facility,Health Facility UID,Operional Status,Level,Ownership,Authority,"(2020, NUT: Percentage of children/babies born with low birth weight (<2.5kg), Jan)","(2020, NUT: Percentage of children/babies born with low birth weight (<2.5kg), Feb)","(2020, NUT: Percentage of children/babies born with low birth weight (<2.5kg), Mar)","(2020, NUT: Percentage of children/babies born with low birth weight (<2.5kg), Apr)","(facility, , )","(district, , )","(index_code, , )"
index_code,,,,,,,,,,,,,,,,,
ABIMAbim Hospital,Karamoja,Abim District,Abim Town Council,SVXuVZuvsit,Abim Hospital,RxPKxSBe0DC,Functional,Hospital,GOV,MOH,9.4,20.0,14.0,25.0,Abim Hospital,ABIM,ABIMAbim Hospital
ABIMAbim Obolokome HC II,Karamoja,Abim District,Morulem Subcounty,YeM7sYuP1xa,Abim Obolokome HC II,ltIECAx2ppI,Functional,HC II,GOV,MOH,0.0,0.0,20.0,0.0,Abim Obolokome HC II,ABIM,ABIMAbim Obolokome HC II
ABIMAdea HC II,Karamoja,Abim District,Morulem Subcounty,YeM7sYuP1xa,Adea HC II,vXlnJIdin7f,Functional,HC II,GOV,MOH,NaN,NaN,0.0,0.0,Adea HC II,ABIM,ABIMAdea HC II
ABIMAlerek HC III,Karamoja,Abim District,Alerek Subcounty,un9dv9VS0Cd,Alerek HC III,RLWAwgTDjSJ,Functional,HC III,GOV,MOH,0.0,10.5,0.0,12.0,Alerek HC III,ABIM,ABIMAlerek HC III
ABIMAmita HC II,Karamoja,Abim District,Abim Subcounty,A8Xv1XbkdrA,Amita HC II,iwlSS6Uybvp,Functional,HC II,GOV,MOH,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZOMBOPaidha HC III,West Nile,Zombo District,Paidha Town Council,wMj4Uco2mMl,Paidha HC III,P1F2OCX6d3t,Functional,HC III,GOV,MOH,3.7,3.3,4.4,5.6,Paidha HC III,ZOMBO,ZOMBOPaidha HC III
ZOMBOPamitu HC II,West Nile,Zombo District,Abanga Subcounty,xT38Pbk8s4j,Pamitu HC II,ucuVKnNXEhV,Functional,HC II,GOV,MOH,NaN,NaN,NaN,NaN,Pamitu HC II,ZOMBO,ZOMBOPamitu HC II
ZOMBOTher-Uru HC II,West Nile,Zombo District,Atyak Subcounty,tAM0excypCp,Ther-Uru HC II,mdd2qwn2jWm,Functional,HC II,GOV,MOH,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
lbw_rep_gvt=lbw_gvt.copy().groupby('district').count()

KeyError: 'district'

In [92]:
# Should buid a function out of that 

for m in ['Jan','Feb','Mar','Apr']:
    lbw_rep_all[('2020','lbw reporting',m)]=lbw_rep_all[('2020','NUT: Percentage of children/babies born with low birth weight (<2.5kg)',m)]/lbw_rep_all['facility']

In [93]:
lbw_rep_all.sum(axis=0)/135

2020      NUT: Percentage of children/babies born with low birth weight (<2.5kg)  Jan     8.911111
                                                                                  Feb     9.311111
                                                                                  Mar     9.533333
                                                                                  Apr     9.748148
facility                                                                                 56.562963
2020      lbw reporting                                                           Jan     0.243306
                                                                                  Feb     0.246500
                                                                                  Mar     0.261611
                                                                                  Apr     0.267150
dtype: float64

## Get that to csv

In [234]:
sam_export_df=sam_df.copy()
sam_export_df.columns=sam_df.columns.map("_".join)
sam_export_df.to_csv(output_path+'sam_map.csv')

Note that I am not looking at old data here, because I coul not find teh corresponding indicators execpt for two of them